In [ ]:
import tkinter
from tkinter import *
from tkinter import messagebox
from tkinter import ttk
from tkinter import filedialog
import tkinter as tk
import pandas as pd
import time


In [ ]:
df_r1 = pd.DataFrame()
df1 = {}
phone_type = []
consumer_type = []
ss = ''
phone_dict = {}
rule_dict = {}
belong_dict = {}
dict_list_menu = {}
Checkbutton_dict = {}


In [ ]:
phone_type


In [ ]:
phone_dict


In [ ]:
rule_dict


In [ ]:
class Util():
    def openFilePath_r():
        global datapath_r
        global df_r1
        global df1
        global ss
        if not df1:
            messagebox.showinfo('提示', '请先导入号码池')
            return
        else:
            print('正在打开……')

            root = tk.Tk()
            root.withdraw()

            datapath_r = filedialog.askopenfilename()
            sss = StringVar(value='正在打开规则')

            Lb5 = Label(win, text='11', textvariable=sss, font=("黑体", 15))
            Lb5.place(x=90, y=650)
            start_time=time.perf_counter()
            df_r1 = pd.read_excel(datapath_r, sheet_name=None)

            end_time=time.perf_counter()
            spent_time=end_time-start_time

            
            sss.set('导入规则成功,耗时:'+str(spent_time)+'s')
            dp = []
            for v1 in df_r1.keys():
                col = df_r1[v1].columns
                # v1代表人员
                for v2 in col:
                    if v2 in phone_type:
                        dp.append(v2)
                rule_dict[v1] = dp
                dp = []

            Rule()
            messagebox.showinfo('提示', '导入规则成功')

        return

    def openFilePath():
        global datapath1
        global df1
        global ss
        print('正在打开……')
        root = tk.Tk()
        root.withdraw()
        datapath1 = filedialog.askopenfilename()
        # 渲染ListB
        ss = StringVar(value='打开中号码池，预计2-5分钟')

        Lb4 = Label(win, text='11', textvariable=ss, font=("黑体", 15))
        Lb4.place(x=90, y=600)
        start_time=time.perf_counter()
        io = pd.io.excel.ExcelFile(datapath1)
        df1 =pd.read_excel(io, sheet_name=None,dtype='str')

        end_time=time.perf_counter()
        spent_time=end_time-start_time
        ss.set('导入号码池成功,耗时:'+str(spent_time)+'s')

        dp = []
        for v1 in df1.keys():
            col = df1[v1].columns
            # v1代表人员
            for v2 in col:
                if '标签' in v2 or '基站成员' in v2:
                    dp.append(v2)
            phone_dict[v1] = dp
            dp = []

        PhonePool()
        messagebox.showinfo('提示', '导入号码池成功')
        return

    def paidan(*args):
        m1 = messagebox
        m1.showinfo("全息派单2.0", "开始派单")
        start_time=time.perf_counter()
        # 渲染进度在RadionButton下方
        divide()
        end_time=time.perf_counter()
        spent_time=end_time-start_time
        Lb6 = Label(win, text='派单完成,耗时:'+str(spent_time)+'s', font=("黑体", 15))
        Lb6.place(x=90, y=700)
        messagebox.showinfo('完成', '派单完成,耗时:'+str(spent_time)+'s')

    def quit():
        win.destroy()


In [ ]:
rule_dict


In [ ]:
dict_list_menu


In [ ]:
consumer_type


In [ ]:
# p_type consumer
def divide():
    global df1
    global df_r1

    df_res_total_l = pd.DataFrame()
    for consumer in consumer_type:
        tmp = rule_dict[consumer]
        for p_type in tmp:
            # print(consumer)
            # print(df_r1[consumer][type])
            df_r = df_r1[consumer]

            # df_r.reset_index()
            df_r = pd.DataFrame(df_r, dtype=str)
            # print(type(df_r))#1

            # 派单量
            for index, row in df_r.iterrows():
                # print(df1[p_type])#我号异宽号码池

                # print(p_type)
                r_id = dict_list_menu[consumer][2].get()

                df_origin = df1[p_type]
                # print(type(row[r_id])

                df_tmp = df_origin[df_origin[r_id] == str(row[r_id])]

                code = row['派单编号']

                pdl = int(row[p_type])

                df_tmp = df_tmp.sort_values(
                    by=phone_dict[p_type], ascending=False)
                if pdl <= len(df_tmp):
                    df_res = df_tmp.head(pdl)
                    df_res['派单编号'] = code
                else:
                    df_res = df_tmp.head(len(df_tmp))
                    df_res['派单编号'] = code
                df1[p_type] = df1[p_type][~df1[p_type].号码.isin(df_res.号码)]
                df_res_total_l = pd.concat([df_res_total_l, df_res])
            df_res_total_l.to_excel(consumer+'-'+p_type+'-'+'结果.xlsx')

            df1[p_type] = df1[p_type][~df1[p_type].号码.isin(df_res_total_l.号码)]
            df_res_total_l = pd.DataFrame()


In [ ]:

def getPhoneBoxValue(*args):
    global df1
    q = lb.curselection()
    print('1进入')
    # lists为当前选择值
    lists = lb.get(q)

    col_list = df1[lists].columns

    res_list = []
    for v in col_list:
        if '标签' in v or '基站成员' in v:
            res_list.append(v)

    var3.set(res_list)
    return


def getRuleBoxValue(*args):
    global lb2
    global df_r1
    q = lb2.curselection()
    print('2进入')
    # lists为当前选择值
    lists = lb2.get(q)
    print(lists)
    col_list = df_r1[lists].columns

    res_list = []
    for v in col_list:
        # print(v)
        # print(phone_type)
        if v in phone_type:
            res_list.append(v)
    # print(res_list)

    var5.set(res_list)
    return


def PhonePool():
    global phone_type
    global lb
    global lb1
    global var2
    global var3
    phone_type = []
    L1 = Label(win, text='号码池种类', font=("黑体", 15))
    L1.place(x=90, y=83)
    var2 = tk.StringVar()  # 存储可迭代对象的值
    var3 = tk.StringVar()
    for v in df1.keys():
        phone_type.append(v)

    var2.set(phone_type)  # 列表也可，设置初始值

    lb = tk.Listbox(win, listvariable=var2,
                    exportselection=False, width=15, height=10)
    lb.place(x=90, y=130)
    lb1 = tk.Listbox(win, listvariable=var3,
                     exportselection=False, width=15, height=10)
    lb1.place(x=200, y=130)
    # 为第一个Listbox设置绑定事件
    lb.bind("<<ListboxSelect>>", getPhoneBoxValue)
    return


def Rule():
    global consumer_type
    global lb2
    global lb3
    global var4
    global var5
    global dict_list_menu
    global Checkbutton_dict
    L2 = Label(win, text='派单人员', font=("黑体", 15))
    L2.place(x=380, y=83)
    var4 = tk.StringVar()  # 存储可迭代对象的值
    var5 = tk.StringVar()
    for v in df_r1.keys():
        consumer_type.append(v)

    var4.set(consumer_type)  # 列表也可，设置初始值

    # print(consumer_type)
    lb2 = tk.Listbox(win, listvariable=var4,
                     exportselection=False, width=15, height=10)
    lb2.place(x=380, y=130)
    lb3 = tk.Listbox(win, listvariable=var5,
                     exportselection=False, width=15, height=10)
    lb3.place(x=490, y=130)
    # 为第一个Listbox设置绑定事件
    lb2.bind("<<ListboxSelect>>", getRuleBoxValue)

    L3 = Label(win, text='选择归属', font=("黑体", 15))
    L3.place(x=90, y=350)

    count = 0
    feet = 90
    dp_list = []
    for i in range(0, len(phone_type)-1):
        dp_list.append(StringVar(value=1))

    dict_list_menu = {}

    for v in consumer_type:
        # v我号异宽
        if count < len(consumer_type):
            Label(win, text=v, textvariable=v, font=(
                "黑体", 15)).place(x=feet, y=400)
            if '渠道' in v:
                In1 = StringVar(value='网格ID')
            else:
                In1 = StringVar(value='服务中心ID')
            L1 = Radiobutton(win, text='服务中心ID', variable=In1, value='服务中心ID',
                             height=5,
                             width=20)
            L1.place(x=feet-40, y=430)
            L2 = Radiobutton(win, text='网格ID', variable=In1, value='网格ID',
                             height=5,
                             width=20)
            L2.place(x=feet-50, y=500)
            dict_list_menu[v] = [L1, L2, In1]
            feet += 150
            count += 1
    count = 0

    return


Button 初始化渲染


In [ ]:
def Button_Init():
    global Lb4
    B1 = tkinter.Button(win, text="导入号码池", command=Util.openFilePath)
    B1.place(x=150, y=20)
    B2 = tkinter.Button(win, text="导入派单量", command=Util.openFilePath_r)
    B2.place(x=300, y=20)
    B3 = tkinter.Button(win, text="开始派单", command=Util.paidan)
    B3.place(x=450, y=20)
    B4 = tkinter.Button(win, text="退出", command=Util.quit)
    B4.place(x=600, y=20)


In [ ]:
# 创建主窗口
win = tkinter.Tk()
# 设置标题
win.title("全息派单")
# 设置大小和位置
win.geometry("800x800+200+20")

# 按钮渲染
Button_Init()


# 显示出来
win.mainloop()


In [ ]:
dict_list_menu


In [ ]:
phone_dict


In [ ]:
rule_dict


In [ ]:
phone_type
